In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [8]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()
print env.P[1][0]
print env.P[1][1]

[(1.0, 1, -1.0, False)]
[(1.0, 2, -1.0, False)]


In [49]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI environment. env.P represents the transition probabilities of the environment.
        theta: Stopping threshold. If the value of all states changes less than theta
            in one iteration we are done.
        discount_factor: lambda time discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    

    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    i = 0
    while True:
        i += 1
        V2 = np.zeros(env.nS)
        for state in env.P:
            v = np.zeros(policy[state].shape)
            #print v
            for action in env.P[state]:
                #v = np.zeros(policy[state])
                for next_state_prob, next_state, reward, done in env.P[state][action]:
                    v[action] += next_state_prob * (reward + np.array(V)[next_state])
                #print v
            V2[state] = np.max(v)
            #print V2
        print np.all((V-V2) < theta)
        if np.all((V-V2) < theta):
            break
        #if i > 2:
        #    break
        V = V2
    for state in env.P:
        
        next_actions_states = {action:env.P[state][action][0][1] for action in env.P[state]}
        
        next_states_values = np.array(V)[next_actions_states.values()]
        
        next_action = np.argmax(next_states_values)
        policy[state][next_action]=1
    
    return policy, V

In [50]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

False
False
False
True
Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [51]:
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)